# Stock Price Forecasting with Long Short-Term Memory Network and Feature Extraction by Autoencoders

## Libraries

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

## Data Preparation

In [4]:
ibov = yf.Ticker('^BVSP')
ibov_data_raw = ibov.history(start='2023-11-17', end='2023-11-24', interval='1m')
ibov_data_raw

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-11-17 10:00:00-03:00,124639.242188,124639.242188,124639.242188,124639.242188,0,0.0,0.0
2023-11-17 10:01:00-03:00,124639.242188,124708.648438,124639.242188,124708.648438,0,0.0,0.0
2023-11-17 10:02:00-03:00,124754.578125,124773.359375,124754.578125,124773.359375,0,0.0,0.0
2023-11-17 10:03:00-03:00,124783.109375,124783.109375,124781.578125,124781.578125,0,0.0,0.0
2023-11-17 10:04:00-03:00,125094.859375,125117.210938,125094.859375,125117.210938,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-23 16:55:00-03:00,126580.789062,126580.789062,126572.703125,126572.703125,0,0.0,0.0
2023-11-23 16:56:00-03:00,126588.546875,126588.546875,126574.593750,126574.593750,0,0.0,0.0
2023-11-23 16:57:00-03:00,126582.921875,126582.921875,126561.492188,126561.492188,0,0.0,0.0


## References

A deep learning framework for financial time series using stacked autoencoders and long-short term memory 

## Draft